In [20]:
%pip install google-cloud-mlflow

  Using cached mlflow-1.30.1-py3-none-any.whl.metadata (11 kB)
  Using cached importlib_metadata-5.2.0-py3-none-any.whl.metadata (5.0 kB)
Using cached mlflow-1.30.1-py3-none-any.whl (17.0 MB)
Using cached importlib_metadata-5.2.0-py3-none-any.whl (21 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 13.8 MB/s eta 0:00:00
  Attempting uninstall: sqlalchemy
    Found existing installation: SQLAlchemy 1.3.13
    Uninstalling SQLAlchemy-1.3.13:
      Successfully uninstalled SQLAlchemy-1.3.13
  Attempting uninstall: importlib-metadata
    Found existing installation: importlib_metadata 8.4.0
    Uninstalling importlib_metadata-8.4.0:
      Successfully uninstalled importlib_metadata-8.4.0
  Attempting uninstall: mlflow
    Found existing installation: mlflow 1.11.0
    Uninstalling mlflow-1.11.0:
      Successfully uninstalled mlflow-1.11.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the

In [21]:
import mlflow
from mlflow.deployments import get_deploy_client
from sklearn.model_selection import train_test_split
from sklearn.datasets import load_diabetes 
from sklearn.ensemble import RandomForestRegressor
import pandas as pd
import numpy as np

import sklearn
print(sklearn.__version__)
from sklearn.metrics import make_scorer

1.5.2


In [22]:
# load dataset
db = load_diabetes()
X = db.data
y = db.target
X_train, X_test, y_train, y_test = train_test_split(X, y)
 
# mlflow.sklearn.autolog() requires mlflow 1.11.0 or above.
# mlflow.sklearn.autolog()
 
# With autolog() enabled, all model parameters, a model score, and the fitted model are automatically logged.  
with mlflow.start_run() as run:  
  # Set the model parameters. 
  n_estimators = 100
  max_depth = 6
  max_features = 3
  # Create and train model.
  rf = RandomForestRegressor(n_estimators = n_estimators, max_depth = max_depth, max_features = max_features)
  rf.fit(X_train, y_train)
  # Use the model to make predictions on the test dataset.
  predictions = rf.predict(X_test)
  
mlflow.end_run()

2024/09/16 00:49:23 WARNING mlflow.utils.environment: Encountered an unexpected error while inferring pip requirements (model URI: /var/folders/xd/vmrgp95576bb5rz_w_ttrvdc0000gn/T/tmptplsfhxp/model/model.pkl, flavor: sklearn), fall back to return ['scikit-learn==1.5.2', 'cloudpickle==2.2.1']. Set logging level to DEBUG to see the full traceback.
2024/09/16 00:49:23 WARNING mlflow.utils.autologging_utils: Encountered unexpected error during sklearn autologging: No module named 'distutils'


In [23]:
model_name = "vertex-sklearn-blog-demo"
mlflow.sklearn.log_model(rf, model_name, registered_model_name=model_name)

2024/09/16 00:49:24 WARNING mlflow.utils.environment: Encountered an unexpected error while inferring pip requirements (model URI: /var/folders/xd/vmrgp95576bb5rz_w_ttrvdc0000gn/T/tmp8dw693cj/model/model.pkl, flavor: sklearn), fall back to return ['scikit-learn==1.5.2', 'cloudpickle==2.2.1']. Set logging level to DEBUG to see the full traceback.


ModuleNotFoundError: No module named 'distutils'

In [ ]:
client = mlflow.tracking.MLflowClient()
model_version_infos = client.search_model_versions(f"name = '{model_name}'")
model_version = max([int(model_version_info.version) for model_version_info in model_version_infos])
model_uri=f"models:/{model_name}/{model_version}"

# model_uri should be models:/vertex-sklearn-blog-demo/1

AttributeError: module 'mlflow.tracking' has no attribute 'MLflowClient'

In [ ]:
# Really simple Vertex client instantiation
vtx_client = mlflow.deployments.get_deploy_client("google_cloud")
deploy_name = f"{model_name}-{model_version}"

# Deploy to Vertex AI using three lines of code! Note: If using python > 3.7, this may take up to 20 minutes.
deployment = vtx_client.create_deployment(
    name=deploy_name,
    model_uri=model_uri)

In [ ]:
# Use the .predict() method from the same plugin
predictions = vtx_client.predict(deploy_name, X_test)

In [ ]:
mlflow.end_run()